# Day 11: Space Police
In this chapter of the `Intcode` saga, we are using our Intcode computer to paint our spaceship.

First we'll copy our Intcode computer code below, with some slight changes.

In [273]:
from collections import defaultdict

In [247]:
class Instruction:
    def __init__(self, opcode, length, param_locations, param_values, memory):
        self.opcode = opcode
        self.length = length
        self.param_locations = param_locations
        self.param_values = param_values
        self.memory = memory
        
    def get_val(self, i):
        return self.param_values[i]
    
    def set_val(self, i, val):
        self.memory[self.param_locations[i]] = val

In [248]:
class Add(Instruction):
    def call(self):
        self.set_val(2, self.get_val(0) + self.get_val(1))

In [249]:
class Multiply(Instruction):
    def call(self):
        self.set_val(2, self.get_val(0) * self.get_val(1))

In [250]:
class Input(Instruction):
    def call(self, input_value):
        self.set_val(0, input_value)

In [251]:
class Output(Instruction):
    def call(self):
        return self.get_val(0)

In [252]:
class JumpIfTrue(Instruction):
    def call(self):
        return self.get_val(1) if self.get_val(0) else None

In [253]:
class JumpIfFalse(Instruction):
    def call(self):
        return self.get_val(1) if not self.get_val(0) else None

In [254]:
class LessThan(Instruction):
    def call(self):
        val = 1 if self.get_val(0) < self.get_val(1) else 0
        self.set_val(2, val)

In [255]:
class Equals(Instruction):
    def call(self):
        val = 1 if self.get_val(0) == self.get_val(1) else 0
        self.set_val(2, val)

In [256]:
class RelativeBaseOffset(Instruction):
    def call(self):
        return self.get_val(0)

In [257]:
class Halt(Instruction):
    pass

The one change we make to the `IntcodeComputer` class is the addition of a `queue_input` method. This will let us append new inputs to our program on the fly.

In [258]:
class IntcodeComputer:
    # Instruction opcodes
    ADD = 1
    MULTIPLY = 2
    INPUT = 3
    OUTPUT = 4
    JUMP_IF_TRUE = 5
    JUMP_IF_FALSE = 6
    LESS_THAN = 7
    EQUALS = 8
    RELATIVE_BASE_OFFSET = 9
    HALT = 99
    
    LENGTHS = {
        ADD: 4,
        MULTIPLY: 4,
        INPUT: 2,
        OUTPUT: 2,
        JUMP_IF_TRUE: 3,
        JUMP_IF_FALSE: 3,
        LESS_THAN: 4,
        EQUALS: 4,
        RELATIVE_BASE_OFFSET: 2,
        HALT: 1,
    }
    
    INSTRUCTIONS = {
        ADD: Add,
        MULTIPLY: Multiply,
        INPUT: Input,
        OUTPUT: Output,
        JUMP_IF_TRUE: JumpIfTrue,
        JUMP_IF_FALSE: JumpIfFalse,
        LESS_THAN: LessThan,
        EQUALS: Equals,
        RELATIVE_BASE_OFFSET: RelativeBaseOffset,
        HALT: Halt,
    }
    
    POSITION_MODE = '0'
    IMMEDIATE_MODE = '1'
    RELATIVE_MODE = '2'
    
    def __init__(self, program):
        self.memory = program.copy()
        self.memory_pointer = 0
        self.relative_base = 0
        self.inputs = []
        
    def create_instruction(self):
        op = str(self.memory[self.memory_pointer])
        opcode = int(op[-2:])    
        length = self.LENGTHS[opcode]
        args = [self.memory[self.memory_pointer + i] for i in range(1, length)]
        modes = op[:-2].zfill(length - 1)[::-1]
        locs = []
        vals = []
        for arg, mode in zip(args, modes):
            if mode == self.POSITION_MODE: 
                locs.append(arg)
                vals.append(self.memory[arg])
            elif mode == self.IMMEDIATE_MODE:
                locs.append(None)
                vals.append(arg)
            elif mode == self.RELATIVE_MODE:
                locs.append(arg + self.relative_base)
                vals.append(self.memory[arg + self.relative_base])

        return self.INSTRUCTIONS[opcode](opcode, length, locs, vals, self.memory)
    
    def queue_input(self, input_value):
        self.inputs.append(input_value)
        
    def run(self):
        while True:
            instruction = self.create_instruction()
            if instruction.opcode == self.HALT:
                break
            elif instruction.opcode == self.INPUT:
                value = instruction.call(self.inputs.pop(0))
            else:
                value = instruction.call()
            
            self.memory_pointer += instruction.length
            if instruction.opcode in [self.JUMP_IF_TRUE, self.JUMP_IF_FALSE] and value is not None:
                self.memory_pointer = value
            elif instruction.opcode == self.RELATIVE_BASE_OFFSET:
                self.relative_base += value
            elif instruction.opcode == self.OUTPUT:
                yield value

    @staticmethod
    def to_program(data):
        return defaultdict(int, enumerate(data.copy()))

The majority of new code for this puzzle is our `HullPaintingRobot` class.

This robot takes an Intcode `computer` and an optional `start_color` as its inputs. Then with the `run` method it proceeds to paint the ship's hull.

In this method, we supply input to the `computer` (using our handy new `queue_input` method) in the form of the color of the current hull location we're on.

We then receive two outputs from the computer: the new color to paint, and the direction to turn when we're done. Since our `IntcodeComputer`'s `OUTPUT` instruction simply `yields` a value to our `robot` caller, we can proceed through this back-and-forth between `robot` and `computer` seamlessly until the `computer` halts.

We use complex numbers for making our left/right turning logic concise.

In [259]:
class HullPaintingRobot:
    LEFT = 0
    RIGHT = 1
    
    TURN = {
        LEFT: 1j,
        RIGHT: -1j
    }
    
    BLACK = 0
    WHITE = 1
    
    COLOR_CHAR = {
        BLACK: '.',
        WHITE: '#'
    }
    
    def __init__(self, computer, start_color = BLACK):
        self.computer = computer
        self.hull_colors = defaultdict(int)
        self.hull_counts = defaultdict(int) 
        self.location = 0 + 0j
        self.orientation = 1j
        
        self.hull_colors[self.location] = start_color
        self.computer.queue_input(start_color)
        
    def turn(self, direction):
        self.orientation *= self.TURN[direction]
        self.location += self.orientation
        
    def paint(self, color):
        self.hull_colors[self.location] = color
        self.hull_counts[self.location] += 1
        
    def print_hull(self):
        min_x = min([int(loc.real) for loc in self.hull_colors])
        max_x = max([int(loc.real) for loc in self.hull_colors])
        min_y = min([int(loc.imag) for loc in self.hull_colors])
        max_y = max([int(loc.imag) for loc in self.hull_colors])
        for y in range(max_y, min_y - 1, -1):
            row = [self.COLOR_CHAR[self.hull_colors[complex(x, y)]] for x in range(min_x, max_x + 1)]
            print("".join(row))
        
    def run(self):
        is_paint_mode = True
        for output in self.computer.run():
            if is_paint_mode:
                self.paint(color = output)
            else:
                self.turn(direction = output)
                self.computer.queue_input(self.hull_colors[self.location])

            is_paint_mode = not is_paint_mode

## Part 1
For the first part, we just need to keep track of how many times the `robot` paints something. We keep track of this in its `hull_counts` dict.

In [265]:
with open('day_11.txt') as f:
    data = [int(token) for token in f.readlines()[0].strip().split(',')]

In [271]:
program = IntcodeComputer.to_program(data)
computer = IntcodeComputer(program)
robot = HullPaintingRobot(computer)
robot.run()

In [272]:
len(robot.hull_counts.keys())

2336

## Part 2
For the second part, we actually need to print the result of the paint job. To do this we'll use the `robot`'s `print_hull` method, taking care to flip the vertical axis since we're starting from top to bottom.

In [269]:
program = IntcodeComputer.to_program(data)
computer = IntcodeComputer(program)
robot = HullPaintingRobot(computer, start_color = HullPaintingRobot.WHITE)
robot.run()

In [270]:
robot.print_hull()

.#..#.####..##..####.#..#.###..#....###....
.#..#....#.#..#.#....#.#..#..#.#....#..#...
.#..#...#..#..#.###..##...###..#....#..#...
.#..#..#...####.#....#.#..#..#.#....###....
.#..#.#....#..#.#....#.#..#..#.#....#......
..##..####.#..#.####.#..#.###..####.#......
